In [1]:
from queue import PriorityQueue
import copy
import random

In [23]:
#-----------------------------------------------------PUZZLE MAKING FUNCTIONS-----------------------------------------------------


#Generate solved puzzle of given size
def solved(size):
    #Make puzzle
    puzzle = []
    for r in range(size):
        #make rowS
        row = []
        for c in range(size):
            #insert tile into row
            row.append((r*size)+c+1)
        #insert row into puzzle
        puzzle.append(row)
    #make sure to include empty position
    puzzle[size-1][size-1] = 0
    return puzzle


#allow user to enter puzzle state of given size
def user(size):
    #make puzzle
    puzzle = []
    for r in range(size):
        #make row
        row = []
        for c in range(size):
            #get user input and insert tile into row
            row.append(int(input(f"Tile at row {r} and column {c}:")))
        #insert row into puzzle
        puzzle.append(row)
    return puzzle


#allow for random puzzles to be created
def rand(size, depth):
    r = solved(size)
    numMoves = random.randint(0,depth)
    for move in range(numMoves):
        c = children(r)
        r = c[random.randint(0,len(c)-1)]
    return r


#execute move on puzzle
def move(puzzle, direction):
    zero = find(puzzle,0)
    puzzle[zero[0]][zero[1]] = puzzle[zero[0]+direction[0]][zero[1]+direction[1]]
    puzzle[zero[0]+direction[0]][zero[1]+direction[1]] = 0


#get all valid children of a puzzle
def children(puzzle):
    children = []
    #find the empty position of the puzzle
    zero = find(puzzle, 0)
    #iterate over all moves
    for move in [(1,0),(0,1),(-1,0),(0,-1)]:
        #check if the move goes out of the range of the puzzle
        if 0<=zero[0]+move[0]<len(puzzle) and 0<=zero[1]+move[1]<len(puzzle):
            #make sure to make a deep copy of the puzzle list
            child = copy.deepcopy(puzzle)
            #swap zero and neighbour
            child[zero[0]][zero[1]] = child[zero[0]+move[0]][zero[1]+move[1]]
            child[zero[0]+move[0]][zero[1]+move[1]] = 0
            #add child to list of children
            children.append(child)
    return children

In [24]:
#--------------------------------------------------------UTILITY FUNCTIONS--------------------------------------------------------


#calculate manhattan distance between two pairs
def manhattanDistance(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])


#find given tile in puzzle and return pair (r,c)
def find(puzzle, tile):
    #iterate over rows
    for r in range(len(puzzle)):
        #iterate over columns
        for c in range(len(puzzle[r])):
            t = puzzle[r][c]
            #check for found tile
            if(t == tile):
                return (r,c)
    return (-1,-1)

In [25]:
#--------------------------------------------------------HEURISTIC FUNCTIONS------------------------------------------------------


#simply return 0
def uniformCostHeuristic(current, goal):
    return 0


#return the number of misplaced tiles between the two puzzles
def misplacedTileHeuristic(current, goal):
    #to start, no tiles are misplaced
    count = 0
    #iterate over all tiles
    for t in range(len(current)**2-1):
        #find tile in current puzzle
        c = find(current, t)
        #find tile in goal puzzle
        g = find(goal, t)
        #use boolean value coercement to 1 to add if mismatched
        count += c!=g
    return count


#return manhattan distance between two puzzles
def manhattanDistanceHeuristic(current, goal):
    #to start, distance is 0
    distance = 0
    for t in range(len(current)**2-1):
        #find tile in current puzzle
        c = find(current, t)
        #find tile in goal puzzle
        g = find(goal, t)
        #accumulate distance
        distance += manhattanDistance(c,g)
    return distance

In [75]:
#-------------------------------------------------------SEARCH FUNCTIONS-------------------------------------------------------


#general search function
def generalSearch(initial, goal, heuristic):
    #construct priority queue
    nodes = PriorityQueue()
    #construct list to keep track of visited states
    visited = []
    #insert inital state into queue with depth 0, priority 0, and path of itself
    nodes.put((0,[0,initial,[initial]]))
    while True:
        #return failure if no nodes left
        if nodes.empty():
            return "FAILURE"
        #get and remove top node
        node = nodes.get()
        depth = node[1][0]
        puzzle = node[1][1]
        path = node[1][2]
        visited.append(puzzle)
        #add node to visited
        #return node if puzzle is goal state
        if puzzle == goal:
            return node
        #expand to all children
        for child in children(puzzle):
            #if the child is not visited, insert it into the queue to be visited
            if child not in visited:
                priority = depth+1+heuristic(child,goal)
                newdepth = depth+1
                newpath = copy.deepcopy(path)
                newpath.append(child)
                nodes.put((priority,[newdepth,child,newpath]))

In [76]:
test = [
    [[1,2,3],[4,5,6],[7,8,0]],
    None,
    [[1,2,3],[4,5,6],[0,7,8]],
    None,
    [[1,2,3],[5,0,6],[4,7,8]],
    None,
    None,
    None,
    [[1,3,6],[5,0,2],[4,7,8]],
    None,
    None,
    None,
    [[1,3,6],[5,0,7],[4,8,2]],
    None,
    None,
    None,
    [[1,6,7],[5,0,3],[4,8,2]],
    None,
    None,
    None,
    [[7,1,2],[4,8,5],[6,3,0]],
    None,
    None,
    None,
    [[0,7,2],[4,6,1],[3,5,8]]
]

In [77]:
def testCases(heuristic):
    goal = solved(3)
    result = []
    for puzzle in test:
        if puzzle == None:
            result.append(None)
        else:
            result.append(len(generalSearch(puzzle,goal,heuristic)[1][2])-1)
    for depth in range(len(result)):
        if result[depth] != None:
            if depth == result[depth]:
                print(f"PASSED DEPTH {depth}")
            else:
                print(f"FAILED DEPTH {depth} - RESULT DEPTH AT {result[depth]}")

In [78]:
testCases(manhattanDistanceHeuristic)

PASSED DEPTH 0
PASSED DEPTH 2
PASSED DEPTH 4
PASSED DEPTH 8
PASSED DEPTH 12
PASSED DEPTH 16
PASSED DEPTH 20
PASSED DEPTH 24


In [79]:
testCases(misplacedTileHeuristic)

PASSED DEPTH 0
PASSED DEPTH 2
PASSED DEPTH 4
PASSED DEPTH 8
PASSED DEPTH 12
PASSED DEPTH 16
PASSED DEPTH 20
PASSED DEPTH 24
